<a href="https://colab.research.google.com/github/Smartsie/Playground/blob/master/Char_RNN_Thierry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
This code is just for fun to make a text generator by myself, inspired by Karpathy but completely made up by myself
T Martin, 12/11/2021
"""

'\nThis code is just for fun to make a text generator by myself, inspired by Karpathy but completely made up by myself\nT Martin, 12/11/2021\n'

In [8]:
import numpy as np
import pandas as pd
import urllib

In [30]:
# Import the big text data
url = "http://f8cho.free.fr/RABELAIS.txt"


In [31]:
import requests
from bs4 import BeautifulSoup


def get_data(url):
    r =  requests.get(url)
    if r.status_code == 200:
        return r.content

data = get_data(url)

In [32]:
data

b'RABELAIS\r\n\r\n1483 - 1553\r\n\r\nArtiste et \xe9crivain\r\nFrancais)\r\nN\xe9 en 1483 (approximativement)\r\nD\xe9c\xe9d\xe9 le 07 janvier 1553\r\n(\xe0 l\'\xe2ge d\'environ 70 ans)\r\n\r\nRabelais est n\xe9 \xe0 Chinon. 0n ne sait pas la date pr\xe9cise de sa naissance, qu\'on \xe0 fix\xe9e en 1483, en 1490 et en 1495 La date de 1490, donn\xe9e par Guy Patin,\r\nest celle qui nous para\xeet devoir \xeatre adopt\xe9e de pr\xe9f\xe9rence. 1519-1524, L\'existence de Rabelais est saisie pour la premi\xe8re fois dans un document contemporain\r\nen 1519.Un acte d\' achat de la moiti\xe9 d\' une auberge \xe0 Fontenay-le-C\xf4mte, par les religieux franciscains du couvent de cette ville, pass\xe9 le 5 avril de\r\ncette ann\xe9e, porte les signatures d\' une douzaine de moines et entre autres, celle de Fran\xe7ois Rabelais. Ainsi le futur auteur Gargantua et de Pantagruel\r\nnous appara\xeet d\' abord sous le froc du fr\xe8re mineur ou du cordelier. Par sa supplique au pape Paul Ill, Rabel

In [33]:
text=requests.get(url).content

In [34]:
text[12000:12050]

b'ent, le 23 f\xe9vrier, sur la question de lui donner '

In [35]:
Encoding(text)

NameError: ignored